<a href="https://colab.research.google.com/github/adityatiwari18/Sentiment140-dataset-Analysis/blob/master/SentimentAnaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing Necessary Libraries**

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [0]:
import os

In [0]:
import string
import re

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd /content/gdrive/My Drive/Sentiment Analysis

/content/gdrive/My Drive/Sentiment Analysis


In [0]:
import Filter #importing filter.py python script

## **Loading Sentiment140 Dataset**

In [0]:
dataset = pd.read_csv('dataset.csv', encoding='latin-1')

In [0]:
dataset.head()

,Sentiment,ID,Date,Status,Name,Review
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [0]:
dataset = shuffle(dataset)

In [0]:
Inputs = dataset.iloc[:,-1].tolist()
Labels = dataset.iloc[:,0:1].to_numpy()

In [0]:
len(Inputs)

1600000

In [0]:
len(Labels)

1600000

## **Preprocessing Data**

In [0]:
#Correcting the labels of the dataset for binary classification (0: negative 1:positive)
for i,(j) in enumerate(Labels):
    if(Labels[i]==4):
        Labels[i]=1

In [0]:
#Removing Usernames, hastags and other useless information from the text from filter.py script
for i,(j) in enumerate(Inputs):
    Filter.tweet = Inputs[i]
    Inputs[i] = Filter.Filter.FilterUsernamesAndURLs()

In [0]:
Inputs[:5]

[' i need my study buddy this isnt working out',
 'i saw joyvey ',
 ' im so happy 4 u ',
 ' lol thankyou but still lol not very impressed with myself at all atm what you been up to',
 'went out driving with jess today we went to the dome ']

In [0]:
#Splitting Data into Train and Test
x_train, x_test, y_train, y_test = train_test_split(Inputs, Labels, test_size = 0.2, random_state = 0)

In [0]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

1280000
1280000
320000
320000


In [0]:
y_train.shape

(1280000, 1)

## **Further Preprocessing (Converting words to vector representation by creating dictionary)**

In [0]:
oov = "<oov>"
dict_length=50000
embed_dims= 12
Embed_input = 228
paddingtype = "post"

In [0]:
tokenizer = Tokenizer(num_words=dict_length, oov_token=oov)
tokenizer.fit_on_texts(x_train)

In [0]:
index = tokenizer.word_index

In [0]:
len(index)

262756

In [0]:
xtrain_sequence = tokenizer.texts_to_sequences(x_train)
xtrain_padded = pad_sequences(xtrain_sequence, padding=paddingtype)
xtest_sequence = tokenizer.texts_to_sequences(x_test)

In [0]:
xtest_padded = pad_sequences(xtest_sequence,maxlen=228 ,padding=paddingtype)

In [0]:
print(xtrain_padded[1])

[178  78  29   2  66 127  10 153 137   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [0]:
set(y_train[0])

{1}

## **Defining Model**

In [0]:
def create_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(dict_length, embed_dims, input_length=Embed_input),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(84, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(68)),
    tf.keras.layers.Dense(42, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  model.summary()
  model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.SGD(), metrics=['accuracy'])

  return model

## **Creating Callback for saving and loading model weights**

In [0]:
path='/content/gdrive/My Drive/Sentiment Analysis/Training/model.ckpt'
dir_path = os.path.dirname(path)
Callback_model = tf.keras.callbacks.ModelCheckpoint(path, save_weights_only=True, verbose=1)

In [0]:
model = create_model()
model.load_weights('/content/gdrive/My Drive/Sentiment Analysis/Training/model.ckpt')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 228, 12)           600000    
_________________________________________________________________
bidirectional (Bidirectional (None, 228, 168)          65184     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 136)               128928    
_________________________________________________________________
dense (Dense)                (None, 42)                5754      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 43        
Total params: 799,909
Trainable params: 799,909
Non-trainable params: 0
_________________________________________________________________


## **Training Model over our preprocessed dataset**

In [0]:
result = model.fit(xtrain_padded, y_train, epochs=20, verbose=1, batch_size=128, validation_data=(xtest_padded, y_test), callbacks = [Callback_model])

Epoch 1/20
 9999/10000 [============================>.] - ETA: 0s - loss: 0.5452 - accuracy: 0.7176
Epoch 00001: saving model to /content/gdrive/My Drive/Sentiment Analysis/Training/model.ckpt
10000/10000 [==============================] - 571s 57ms/step - loss: 0.5452 - accuracy: 0.7176 - val_loss: 0.4950 - val_accuracy: 0.7579
Epoch 2/20
10000/10000 [==============================] - ETA: 0s - loss: 0.4796 - accuracy: 0.7686
Epoch 00002: saving model to /content/gdrive/My Drive/Sentiment Analysis/Training/model.ckpt
10000/10000 [==============================] - 563s 56ms/step - loss: 0.4796 - accuracy: 0.7686 - val_loss: 0.4641 - val_accuracy: 0.7783
Epoch 3/20
10000/10000 [==============================] - ETA: 0s - loss: 0.4582 - accuracy: 0.7831
Epoch 00003: saving model to /content/gdrive/My Drive/Sentiment Analysis/Training/model.ckpt
10000/10000 [==============================] - 559s 56ms/step - loss: 0.4582 - accuracy: 0.7831 - val_loss: 0.4511 - val_accuracy: 0.7871
Epoch 4

## **Evaluating model performance over test dataset**

In [0]:
model.evaluate(xtest_padded, y_test, batch_size=320, verbose=1)

1000/1000 [==============================] - 33s 33ms/step - loss: 0.4111 - accuracy: 0.8113


[0.41109341382980347, 0.8113093972206116]